# 네이버 자주 본 뉴스 크롤링

1) 수집내용

    1) 많이본뉴스–섹션별(정치~IT/과학)Top5기사제목,신문사,뷰 
    2) 해당 기사별 기사 내용, 리액션 (좋아요 ~ 후속기사 원해요)
    
2) 수집방법(택1)

    1) [기본] Requests , BeautifulSoup, Selenium
    2) [심화] Requests, BeautifulSoup (+ 멀티프로세싱)
    
3) 수집범위 및 저장

    1) 2019년7월21일~2020년8월20일(동작가능,실제구동x)
    2) 하나의 파일로 저장 (방식 자유)
    3) Ex)총6섹션*Top5*365일=10950rows

In [76]:
import requests
from urllib.parse import urlparse, parse_qs
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from IPython.display import Image
import os
from pathlib import Path
import glob

import datetime

### 날짜 String 생성

In [77]:
start_date = datetime.datetime(2019, 8, 21, 3, 22, 32)
start_date.strftime("%Y%m%d %Y-%m-%d 시간 표시외 %H:%M:%S 아무거나 넣어도 됩니다.") #strftime 활용 예시

'20190821 2019-08-21 시간 표시외 03:22:32 아무거나 넣어도 됩니다.'

In [78]:
# date를 가져오는 함수 
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [79]:
date_string_list = get_date_string(start_date, 366)

In [80]:
date_string_list[0], date_string_list[-1] # 생성 완료

('20190821', '20200820')

## Part 1. 많이 본 뉴스 TOP5 목록 크롤링

In [81]:
# 6가지 섹션에 대한 TOP5 목록 가져오기
def get_top_news(date): 
    # 많이 본 뉴스 페이지 링크
    url = f"https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&date={date}" 
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    result = []
    # 6가지 섹션에 대해서만 진행
    for section in soup.select('.content .content .ranking_section')[:6]:
        for news in section.select('li'):
            result.append(
                    [
                        # 뉴스의 섹션 정보 ex) '정치'
                        news.parent.parent.select_one('h4').text.strip(),
                        # TOP5에 선정된 날짜
                        date,
                        # 뉴스의 제목 
                        news.select_one('dt a')['title'],
                        # 뉴스의 링크 (news.naver.com을 추가한 링크)
                        'https://news.naver.com' + news.select_one('dt a')['href'],
                        # 뉴스를 작성한 신문사 명
                        news.select_one('span:not(.lede)').text.strip(),
                        # 뉴스 조회수
                        news.select_one('.count_view').text
                    ]
            )
    
    return result

In [82]:
news_data = []
# 2019.08.21 ~ 2020.08.20
for date in date_string_list:
    news_data.extend(get_top_news(date)) 

### news_data 생성

**데이터의 길이**

In [83]:
len(news_data)

10980

**결과 예시**

In [84]:
news_data[:3] # 결과 예시

[['정치',
  '20190821',
  '한국인 관광객 줄었지만…日타격은 미미',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=009&aid=0004413757&date=20190821&type=1&rankingSectionId=100&rankingSeq=1',
  '매일경제',
  '227,243'],
 ['정치',
  '20190821',
  '[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 바로 그만둬',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003468413&date=20190821&type=1&rankingSectionId=100&rankingSeq=2',
  '조선일보',
  '223,896'],
 ['정치',
  '20190821',
  "'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=079&aid=0003261328&date=20190821&type=1&rankingSectionId=100&rankingSeq=3',
  '노컷뉴스',
  '173,525']]

### 데이터프레임 변환

In [85]:
# 결과물을 데이터 프레임으로 변환 및 Column Name 부여
df_top_news = pd.DataFrame(news_data, 
                           columns=["section", "TOP_date", "title", "url", "press", "views"])

In [86]:
df_top_news

,section,TOP_date,title,url,press,views
0,정치,20190821,한국인 관광객 줄었지만…日타격은 미미,https://news.naver.com/main/ranking/read.nhn?m...,매일경제,"227,243"
1,정치,20190821,"[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 ...",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"223,896"
2,정치,20190821,"'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",https://news.naver.com/main/ranking/read.nhn?m...,노컷뉴스,"173,525"
3,정치,20190821,"법무부·高大 ""조국 딸 논문, 대입 미반영"" 거짓말",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"164,972"
4,정치,20190821,"조국 딸 제1저자로 올린 단국대 교수 ""英作에 굉장히 기여...부끄럽지 않다""",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"149,720"
...,...,...,...,...,...,...
10975,IT/과학,20200820,'기안84 '복학왕' 연재 중단' 유저도 들고 일어났다···네이버 결단할까,https://news.naver.com/main/ranking/read.nhn?m...,서울경제,"495,591"
10976,IT/과학,20200820,[사이언스카페] 소변기 물 내려도 코로나 퍼진다,https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"231,893"
10977,IT/과학,20200820,[단독] SK하이닉스 이천사업장서 코로나 확진자 발생,https://news.naver.com/main/ranking/read.nhn?m...,조선비즈,"136,005"
10978,IT/과학,20200820,"""혐오장사 중단하라"" 기안84 논란에 저격 당한 네이버",https://news.naver.com/main/ranking/read.nhn?m...,아시아경제,"118,184"


* 6가지 섹션에 대해 top5 날짜, title, url, press, views 저장

## Part 2. TOP5 뉴스의 상세 정보 크롤링

In [87]:
# 뉴스 하나의 상세 내용(본문, 리액션)을 받아오는 함수
def get_news_detail(news):
    # 상세 정보를 가져올 url 저장
    news_url = news.url

    res = requests.get(news_url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    try:
        # 본문 크롤링
        content = soup.select_one('#articleBodyContents').text.strip() 
        news["content"] = content
        
        # 리액션 크롤링
        parse = urlparse(news_url)
        oid = parse_qs(parse.query)['oid'][0]
        aid = parse_qs(parse.query)['aid'][0]
        # 리액션을 포함하고 있는 링크
        reaction_url = "https://news-like.naver.com/v1/search/contents?"+\
                f"q=NEWS%5Bne_{oid}_{aid}%5D%7CNEWS_SUMMARY%5B{oid}_{aid}%5D%7CNEWS_MAIN%5Bne_{oid}_{aid}%5D"
        res = requests.get(reaction_url)
        result = res.json()
        
        # 5가지 리액션(want, angry, like, warm, sad)에 대해 각각 진행
        for reaction in result['contents'][0]['reactions']:
            news[reaction['reactionType']] = reaction['count']
            
    # 해당 링크에 접속되었을 때 게시글이 사라진 경우   
    except:
        news["content"] = '삭제된 뉴스'

    return news

**결과 예시**

In [88]:
top_news_detail = pd.DataFrame(columns=get_news_detail(df_top_news.iloc[0]).index)
for i in range(5):
    top_news_detail.loc[i] = get_news_detail(df_top_news.iloc[i])

top_news_detail

,section,TOP_date,title,url,press,views,content,want,angry,like,warm,sad
0,정치,20190821,한국인 관광객 줄었지만…日타격은 미미,https://news.naver.com/main/ranking/read.nhn?m...,매일경제,"227,243","7월 日찾은 한국인 7.6%↓중국인 방문은 19% 늘어식약처, 일본산 수입식품방사능...",59,2084,1529,52,33
1,정치,20190821,"[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 ...",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"223,896",서울대 총동창회가 1년 전액 지급조국 법무부 장관 후보자의 딸 조모(28)씨가 20...,202,22286,96,44,29
2,정치,20190821,"'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",https://news.naver.com/main/ranking/read.nhn?m...,노컷뉴스,"173,525","웅동학원 상대로 80억원 '편법소송' 남동생, 채권포기 의사 밝혀남동생, 전처 공동...",68,7918,29,20,10
3,정치,20190821,"법무부·高大 ""조국 딸 논문, 대입 미반영"" 거짓말",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"164,972","高2때 2주 인턴후 병리학 논문 1저자 등재, 이듬해 고대 수시입학부정입학 의혹 일...",163,19085,81,40,18
4,정치,20190821,"조국 딸 제1저자로 올린 단국대 교수 ""英作에 굉장히 기여...부끄럽지 않다""",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"149,720","장영표 단국대 교수, ""아내가 한영외고 학부형 모임… 외국大 진학 위해 빨리 게재되...",63,14003,697,18,15


## Default, MultiThreading, MultiProcessing

In [89]:
import concurrent.futures
from multiprocessing import Pool

**실행 시간 비교를 위한 timer 함수**

In [90]:
from time import time
import functools

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time()
        ret = func(*args, **kwargs)
        print(f"실행시간: {time()-start:0.5f}초")
        return ret
    return wrapper

**뉴스 상세정보 크롤링 함수**

In [91]:
frame = pd.DataFrame(columns=get_news_detail(df_top_news.iloc[0]).index)
def test_func(num):
    frame_news = frame
    # df_top_news : TOP5 목록을 담은 데이터프레임
    # df_top_news를 한 행 씩 get_news_detail함수에 적용하여 frame_news에 추가
    return frame_news.append(get_news_detail(df_top_news.iloc[num]))

### Default

In [92]:
@timer
def get_all_usd_single(end_num):
    frame = pd.DataFrame(columns=get_news_detail(df_top_news.iloc[0]).index)
    # df_top_news의 0번째 행부터 마지막 행까지 test_func를 적용한 뒤 frame에 저장
    for num in range(end_num):
        frame = frame.append(test_func(num))
    return frame

In [93]:
data = get_all_usd_single(len(df_top_news))
print(data.shape)
data

실행시간: 3478.77626초
(10980, 12)


,section,TOP_date,title,url,press,views,content,like,want,angry,sad,warm
0,정치,20190821,한국인 관광객 줄었지만…日타격은 미미,https://news.naver.com/main/ranking/read.nhn?m...,매일경제,"227,243","7월 日찾은 한국인 7.6%↓중국인 방문은 19% 늘어식약처, 일본산 수입식품방사능...",1529,59,2084,33,52
1,정치,20190821,"[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 ...",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"223,896",서울대 총동창회가 1년 전액 지급조국 법무부 장관 후보자의 딸 조모(28)씨가 20...,96,202,22286,29,44
2,정치,20190821,"'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",https://news.naver.com/main/ranking/read.nhn?m...,노컷뉴스,"173,525","웅동학원 상대로 80억원 '편법소송' 남동생, 채권포기 의사 밝혀남동생, 전처 공동...",29,68,7918,10,20
3,정치,20190821,"법무부·高大 ""조국 딸 논문, 대입 미반영"" 거짓말",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"164,972","高2때 2주 인턴후 병리학 논문 1저자 등재, 이듬해 고대 수시입학부정입학 의혹 일...",81,163,19085,18,40
4,정치,20190821,"조국 딸 제1저자로 올린 단국대 교수 ""英作에 굉장히 기여...부끄럽지 않다""",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"149,720","장영표 단국대 교수, ""아내가 한영외고 학부형 모임… 외국大 진학 위해 빨리 게재되...",697,63,14003,15,18
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,IT/과학,20200820,'기안84 '복학왕' 연재 중단' 유저도 들고 일어났다···네이버 결단할까,https://news.naver.com/main/ranking/read.nhn?m...,서울경제,"495,591",기본소득당 젠더특위 등 8개 단체 요구안 제출기안84/서울경제DB[서울경제] 만화가...,1281,202,8773,346,81
10976,IT/과학,20200820,[사이언스카페] 소변기 물 내려도 코로나 퍼진다,https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"231,893","중 연구진 ""화장실 갈 때 마스크 착용 의무화 필요""소변기 물을 내리면 바이러스가 ...",35,42,837,63,8
10977,IT/과학,20200820,[단독] SK하이닉스 이천사업장서 코로나 확진자 발생,https://news.naver.com/main/ranking/read.nhn?m...,조선비즈,"136,005","R&D센터 4층서 근무하던 사무직 직원, 오전 양성판정…해당 층 폐쇄·직원 귀가조치...",35,12,218,62,6
10978,IT/과학,20200820,"""혐오장사 중단하라"" 기안84 논란에 저격 당한 네이버",https://news.naver.com/main/ranking/read.nhn?m...,아시아경제,"118,184",여성단체들이 기안84 웹툰 '복학왕' 연재 중단 요구하는 기자회견을 열고 있다.사진...,316,24,1789,26,7


* 10980개의 데이터를 처리하는데 약 58분 소요

### 50개의 데이터 크롤링에 따른 시간 비교

In [103]:
data = get_all_usd_single(50)
print(data.shape)
data.head()

실행시간: 13.14844초
(50, 12)


,section,TOP_date,title,url,press,views,content,angry,warm,like,want,sad
0,정치,20190821,한국인 관광객 줄었지만…日타격은 미미,https://news.naver.com/main/ranking/read.nhn?m...,매일경제,"227,243","7월 日찾은 한국인 7.6%↓중국인 방문은 19% 늘어식약처, 일본산 수입식품방사능...",2084,52,1529,59,33
1,정치,20190821,"[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 ...",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"223,896",서울대 총동창회가 1년 전액 지급조국 법무부 장관 후보자의 딸 조모(28)씨가 20...,22286,44,96,202,29
2,정치,20190821,"'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",https://news.naver.com/main/ranking/read.nhn?m...,노컷뉴스,"173,525","웅동학원 상대로 80억원 '편법소송' 남동생, 채권포기 의사 밝혀남동생, 전처 공동...",7918,20,29,68,10
3,정치,20190821,"법무부·高大 ""조국 딸 논문, 대입 미반영"" 거짓말",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"164,972","高2때 2주 인턴후 병리학 논문 1저자 등재, 이듬해 고대 수시입학부정입학 의혹 일...",19085,40,81,163,18
4,정치,20190821,"조국 딸 제1저자로 올린 단국대 교수 ""英作에 굉장히 기여...부끄럽지 않다""",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"149,720","장영표 단국대 교수, ""아내가 한영외고 학부형 모임… 외국大 진학 위해 빨리 게재되...",14003,18,697,63,15


* Default 방식으로 50개의 데이터 처리하는데 약 13초 소요

### MultiThreading

In [94]:
@timer
def get_all_usd_by_multi_threads(end_num):
    nums = list(range(end_num))
    # df_top_news의 0번째 행부터 마지막 행까지 40개의 thread로 test_func를 적용한 뒤 frame에 저장
    with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:
        result = executor.map(test_func, nums)
        
    return pd.concat(result)

In [104]:
data = get_all_usd_by_multi_threads(50)
print(data.shape)
data.head()

실행시간: 6.90766초
(50, 12)


,section,TOP_date,title,url,press,views,content,angry,sad,like,warm,want
0,정치,20190821,한국인 관광객 줄었지만…日타격은 미미,https://news.naver.com/main/ranking/read.nhn?m...,매일경제,"227,243","7월 日찾은 한국인 7.6%↓중국인 방문은 19% 늘어식약처, 일본산 수입식품방사능...",2084,33,1529,52,59
1,정치,20190821,"[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 ...",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"223,896",서울대 총동창회가 1년 전액 지급조국 법무부 장관 후보자의 딸 조모(28)씨가 20...,22286,29,96,44,202
2,정치,20190821,"'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",https://news.naver.com/main/ranking/read.nhn?m...,노컷뉴스,"173,525","웅동학원 상대로 80억원 '편법소송' 남동생, 채권포기 의사 밝혀남동생, 전처 공동...",7918,10,29,20,68
3,정치,20190821,"법무부·高大 ""조국 딸 논문, 대입 미반영"" 거짓말",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"164,972","高2때 2주 인턴후 병리학 논문 1저자 등재, 이듬해 고대 수시입학부정입학 의혹 일...",19085,18,81,40,163
4,정치,20190821,"조국 딸 제1저자로 올린 단국대 교수 ""英作에 굉장히 기여...부끄럽지 않다""",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"149,720","장영표 단국대 교수, ""아내가 한영외고 학부형 모임… 외국大 진학 위해 빨리 게재되...",14003,15,697,18,63


* MultiThreading 방식으로 50개의 데이터 처리하는데 약 7초 소요

### MultiProcessing

In [105]:
@timer
def get_all_usd_by_multi_processes(end_num):
    nums = list(range(end_num))
    # df_top_news의 0번째 행부터 마지막 행까지 3개의 process로 test_func를 적용한 뒤 frame에 저장
    with Pool(3) as p:
        return pd.concat(p.map(test_func, nums))

In [106]:
top_news_detail = pd.DataFrame(columns=get_news_detail(df_top_news.iloc[0]).index)
data = get_all_usd_by_multi_processes(50)

실행시간: 5.35505초


In [108]:
print(data.shape)
data.head()

(50, 12)


,section,TOP_date,title,url,press,views,content,angry,sad,like,warm,want
0,정치,20190821,한국인 관광객 줄었지만…日타격은 미미,https://news.naver.com/main/ranking/read.nhn?m...,매일경제,"227,243","7월 日찾은 한국인 7.6%↓중국인 방문은 19% 늘어식약처, 일본산 수입식품방사능...",2084,33,1529,52,59
1,정치,20190821,"[단독] 조국 딸, 서울대 환경대학원 2연속 장학금… 부산대 의전원 합격 다음 날 ...",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"223,896",서울대 총동창회가 1년 전액 지급조국 법무부 장관 후보자의 딸 조모(28)씨가 20...,22286,29,96,44,202
2,정치,20190821,"'편법소송' 논란에 조국 동생, '페이퍼컴퍼니' 사실상 실토",https://news.naver.com/main/ranking/read.nhn?m...,노컷뉴스,"173,525","웅동학원 상대로 80억원 '편법소송' 남동생, 채권포기 의사 밝혀남동생, 전처 공동...",7918,10,29,20,68
3,정치,20190821,"법무부·高大 ""조국 딸 논문, 대입 미반영"" 거짓말",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"164,972","高2때 2주 인턴후 병리학 논문 1저자 등재, 이듬해 고대 수시입학부정입학 의혹 일...",19085,18,81,40,163
4,정치,20190821,"조국 딸 제1저자로 올린 단국대 교수 ""英作에 굉장히 기여...부끄럽지 않다""",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"149,720","장영표 단국대 교수, ""아내가 한영외고 학부형 모임… 외국大 진학 위해 빨리 게재되...",14003,15,697,18,63


* MultiProcessing 방식으로 50개의 데이터 처리하는데 약 5초 소요

# 데이터 저장

> 파일 형태로 크롤링한 데이터를 저장

In [96]:
data.to_csv("top5_news_detail.csv")